In [1]:
#####Latent triggers detection with Hawkes Processes#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z

In [3]:
####データの発生####
##データの設定
k = 10
k_vec = np.repeat(1, k)
hh = 5000
item = 3000
Lambda = np.random.gamma(30.0, 1/0.2, hh)
pt = np.random.poisson(Lambda, hh)
hhpt = np.sum(pt)

In [4]:
##IDとインデックスを設定
#IDを設定
d_id = np.repeat(range(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスを設定
d_list = [i for i in range(hh)]
d_vec = [i for i in range(hh)]
pt_list = [j for j in range(np.max(pt))]
pt_n = np.repeat(0, np.max(pt))
for i in range(hh):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")
    d_vec[i] = np.repeat(1, pt[i])
for j in range(np.max(pt)):
    pt_list[j] = np.array(np.where(pt_id==j)[0], dtype="int")
    pt_n[j] = pt_list[j].shape[0]
max_index = np.array([np.max(d_list[i]) for i in range(hh)])

In [5]:
##アイテムの割当を生成
#セグメント割当を生成
topic = 25
phi = np.random.dirichlet(np.repeat(0.5, item), topic)
theta = np.random.dirichlet(np.repeat(2.5, topic), hh)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(hh)]).reshape(hh, topic), range(topic))

#多項分布からアイテムを生成
item_id1 = np.zeros(hhpt, dtype='int')
for i in range(hh):
    if i%1000==0:
        print(i)
    item_id1[d_list[i]] = np.dot(np.random.multinomial(1, phi[z[i], :], pt[i]), range(item))
    
#インデックスの設定
item_list1 = [j for j in range(item)]
item_vec1 = [j for j in range(item)]
item_n1 = np.repeat(0, item)
for j in range(item):
    item_list1[j] = np.array(np.where(item_id1==j)[0], dtype="int")
    item_vec1[j] = np.repeat(1, item_list1[j].shape[0])
    item_n1[j] = len(item_list1[j])

0
1000
2000
3000
4000


In [6]:
##アイテム購買履歴を定義
#データの設定
get = 25
item_id2 = np.array(np.full((hhpt, get), item), dtype="int16")

#アイテムごとに購買履歴を格納
for i in range(hh):
    x = d_list[i]
    for j in range(x.shape[0]):
        index = np.arange(j-get, j)[::-1]; index = index[index >= 0]
        item_id2[x[j], np.arange(index.shape[0])] = item_id1[x[index]]
Z = ((np.arange(get) + 1)[::-1] / get) * np.array(item_id2!=item, dtype="int")

#インデックスを作成
index_history = [j for j in range(get)]
item_list21 = [i for i in range(item)]
item_list22 = [i for i in range(item)]
d_list2 = [i for i in range(item)]
for j in range(get):
    index_history[j] = np.array(np.where(item_id2[:, j]!=item)[0], dtype="int")
for i in range(item):
    temp_list1 = [j for j in range(get)]
    temp_list2 = [j for j in range(get)]
    for j in range(get):
        temp_list1[j] = np.array(np.where(item_id2[:, j]==i)[0], dtype="int")
        temp_list2[j] = d_id[temp_list1[j]]
    item_list21[i] = temp_list1
    item_list22[i] = np.array(np.unique(np.where(item_id2==i)[0]), dtype="int")
    d_list2[i] = temp_list2

In [7]:
##妥当なパラメータが生成するまで反復
rp = 0
while True:
    rp = rp + 1
    
    ##モデルパラメータの生成
    #事前分布の設定
    alpha11 = 0.3; beta11 = 0.5
    alpha12 = 0.3; beta12 = 0.5
    alpha21 = 0.1; beta21 = 0.7
    alpha22 = 0.1; beta22 = 0.7
    alpha3 = 1.0; beta3 = 4.0

    #モデルパラメータを生成
    theta_u1 = np.random.gamma(alpha11, 1/beta11, k*hh).reshape(hh, k)
    theta_v1 = np.random.gamma(alpha12, 1/beta12, k*item).reshape(item, k)
    theta_u2 = np.random.gamma(alpha21, 1/beta21, k*hh).reshape(hh, k)
    theta_v2 = np.random.gamma(alpha22, 1/beta22, k*item).reshape(item, k)
    gamma = np.random.gamma(alpha3, 1/beta3, hh)
    thetat_u1 = theta_u1.copy(); thetat_v1 = theta_v1.copy()
    thetat_u2 = theta_u2.copy(); thetat_v2 = theta_v2.copy()
    gammat = gamma.copy()
    
    ##応答変数を生成
    #モデルの期待値    
    uv1 = np.dot(theta_u1[d_id, ] * theta_v1[item_id1, ], k_vec)
    gamma_vec = gamma[d_id]; theta_user2 = theta_u2[d_id, ]
    uv2 = np.repeat(0.0, hhpt)
    for j in range(get):
        index = index_history[j]
        uv2[index] += np.dot(theta_user2[index, ] * theta_v2[item_id2[index, j], ], k_vec) * (gamma_vec[index] * Z[index, j])
    mu = uv1 + uv2
    mut = mu.copy()

    #指数分布から応答変数を生成
    y = np.random.exponential(mu, hhpt)
    y_vec = y[:, np.newaxis]
    print([rp, np.round(np.max(y), 1)])
    if np.max(y) < 400:
        break

[1, 562.1]
[2, 735.2]
[3, 503.1]
[4, 490.1]
[5, 527.8]
[6, 673.7]
[7, 477.5]
[8, 427.9]
[9, 386.1]


In [8]:
####マルコフ連鎖モンテカルロ法でパラメータを推定####
##切断指数分布の乱数を生成する関数
def rtexp(gamma, a, b):
    #切断指数分布の乱数を生成
    FA = scipy.stats.expon.cdf(a, scale=gamma)
    FB = scipy.stats.expon.cdf(b, scale=gamma)
    par = scipy.stats.expon.ppf(np.random.uniform(0, 1, a.shape[0])*(FB-FA)+FA, scale=gamma)
    return par

In [9]:
##アルゴリズムの設定
R = 2000
keep = 4
burnin = int(500/keep)
iter = 0
disp = 10
e1 = 0.001
e2 = 0.0025
L = 3

In [10]:
##データの設定
#インデックスの作成
item_get = [j for j in range(get)]
z_get = [j for j in range(get)]
for j in range(get):
    index = index_history[j]
    item_get[j] = item_id2[index, j]
    z_get[j] = Z[index, j]

In [23]:
##事前分布の設定
alpha = 0.25
beta = 0.25
s0 = 1.0
v0 = 1.0

In [24]:
##パラメータの真値
#ガンマ分布のパラメータ
alpha11 = 0.3; beta11 = 0.5
alpha12 = 0.3; beta12 = 0.5
alpha21 = 0.1; beta21 = 0.75
alpha22 = 0.1; beta22 = 0.75
tau = 0.25
    
#モデルパラメータの生成
theta_u1 = thetat_u1.copy()
theta_v1 = thetat_v1.copy()
theta_u2 = thetat_u2.copy()
theta_v2 = thetat_v2.copy()
gamma = gammat.copy()

#モデルの期待値
uv1 = np.dot(theta_u1[d_id, ] * theta_v1[item_id1, ], k_vec)
gamma_vec = gamma[d_id]; theta_user2 = theta_u2[d_id, ]
gamma_dt = np.zeros((hhpt, get))
uv2 = np.repeat(0.0, hhpt)
for j in range(get):
    index = index_history[j]
    gamma_dt[index, j] = gamma_vec[index] * z_get[j]
    uv2[index] += np.dot(theta_user2[index, ] * theta_v2[item_get[j], ], k_vec) * gamma_dt[index, j]
mu = uv1 + uv2

In [25]:
##パラメータの初期値
#ガンマ分布のパラメータ
alpha11 = 1.0; beta11 = 1.5
alpha12 = 1.0; beta12 = 1.5
alpha21 = 0.25; beta21 = 1.0
alpha22 = 0.25; beta22 = 1.0
tau = 0.2

#モデルパラメータの初期値
theta_u1 = np.random.gamma(alpha11, 1/beta11, k*hh).reshape(hh, k)
theta_v1 = np.random.gamma(alpha12, 1/beta12, k*item).reshape(item, k)
theta_u2 = np.random.gamma(alpha21, 1/beta21, k*hh).reshape(hh, k)
theta_v2 = np.random.gamma(alpha22, 1/beta22, k*item).reshape(item, k)
gamma = np.random.gamma(0.5, 4.0, hh)

#モデルの期待値
uv1 = np.dot(theta_u1[d_id, ] * theta_v1[item_id1, ], k_vec)
gamma_vec = gamma[d_id]; theta_user2 = theta_u2[d_id, ]
gamma_dt = np.zeros((hhpt, get))
uv2 = np.repeat(0.0, hhpt)
for j in range(get):
    index = index_history[j]
    gamma_dt[index, j] = gamma_vec[index] * z_get[j]
    uv2[index] += np.dot(theta_user2[index, ] * theta_v2[item_get[j], ], k_vec) * gamma_dt[index, j]
mu = uv1 + uv2

In [26]:
##パラメータの格納用配列
THETA_U1 = np.zeros((hh, k , int(R/keep)))
THETA_V1 = np.zeros((item, k , int(R/keep)))
THETA_U2 = np.zeros((hh, k , int(R/keep)))
THETA_V2 = np.zeros((item, k , int(R/keep)))
GAMMA = np.zeros((int(R/keep), hh))

In [27]:
##対数尤度の基準値
#学習データに1パラメータモデルの対数尤度
LLst = np.sum(scipy.stats.expon.logpdf(x=y, scale=np.mean(y)))
print(LLst)

#学習データに対する真値の対数尤度
uv01 = np.dot(thetat_u1[d_id, ] * thetat_v1[item_id1, ], k_vec)
gamma_vec0 = gammat[d_id]; theta_user0 = thetat_u2[d_id, ]
uv02 = np.repeat(0.0, hhpt)
for j in range(get):
    index = index_history[j]
    uv02[index] += np.dot(theta_user0[index, ] * thetat_v2[item_get[j], ], k_vec) * gamma_vec0[index] * z_get[j]
mu0 = uv01 + uv02
LLbest = np.sum(scipy.stats.expon.logpdf(x=y, scale=mu0))
print(LLbest)

-1836955.9978421093
-1460546.9255841456


In [28]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):

    ##ユーザー特徴行列をサンプリング
    #補助変数を更新
    theta_user1 = theta_u1[d_id, ]
    theta_item1 = theta_v1[item_id1, ]
    uv_dt = theta_user1 * theta_item1
    Lambda = uv_dt / mu[:, np.newaxis]

    #ガンマ分布のパラメータ
    lambda_y = Lambda * y_vec
    W11 = np.zeros((hh, k)); W12 = np.zeros((hh, k))
    for i in range(hh):
        index = d_list[i]
        W11[i, ] = np.dot(lambda_y[index, ].T, d_vec[i])
        W12[i, ] = np.dot(theta_item1[index, ].T, d_vec[i])
    W11 = W11 + alpha; W12 = W12 + beta

    #パラメータをサンプリング
    theta_u1 = np.random.gamma(W11.reshape(-1), 1/W12.reshape(-1), hh*k).reshape(hh, k)
    theta_user1 = theta_u1[d_id, ]
    uv1 = np.dot(theta_user1 * theta_item1, k_vec)


    ##アイテム特徴行列をサンプリング
    #補助変数を更新
    mu = uv1 + uv2
    uv_dt = theta_user1 * theta_item1
    Lambda = uv_dt / mu[:, np.newaxis]

    #ガンマ分布のパラメータ
    lambda_y = Lambda * y_vec
    H11 = np.zeros((item, k)); H12 = np.zeros((item, k))
    for i in range(item):
        index = item_list1[i]
        H11[i, ] = np.dot(lambda_y[index, ].T, item_vec1[i])
        H12[i, ] = np.dot(theta_user1[index, ].T, item_vec1[i])
    H11 = H11 + alpha; H12 = H12 + beta

    #パラメータをサンプリング
    theta_v1 = np.random.gamma(H11.reshape(-1), 1/H12.reshape(-1), item*k).reshape(item, k)
    theta_item1 = theta_v1[item_id1, ]
    uv1 = np.dot(theta_user1 * theta_item1, k_vec)


    ##トリガーのユーザー特徴行列をサンプリング
    #補助変数を更新
    mu = uv1 + uv2
    theta_user2 = theta_u2[d_id, ]
    theta_dt = np.zeros((hhpt, k))
    uv_dt = np.zeros((hhpt, k))
    for j in range(get):
        index = index_history[j]
        theta_target = theta_user2[index, ]
        uv_target = (theta_target * theta_v2[item_get[j], ]) * gamma_dt[index, j][:, np.newaxis]
        theta_dt[index, ] += uv_target / theta_target  
        uv_dt[index, ] += uv_target
    Lambda = uv_dt / mu[:, np.newaxis]

    #ガンマ分布のパラメータ
    lambda_y = Lambda * y_vec
    W21 = np.zeros((hh, k)); W22 = np.zeros((hh, k))
    for i in range(hh):
        index = d_list[i]
        W21[i, ] = np.dot(lambda_y[index, ].T, d_vec[i])
        W22[i, ] = np.dot(theta_dt[index, ].T, d_vec[i])
    W21 = W21 + alpha; W22 = W22 + beta

    #パラメータをサンプリング
    theta_u2 = np.random.gamma(W21.reshape(-1), 1/W22.reshape(-1), hh*k).reshape(hh, k)
    theta_user2 = theta_u2[d_id, ]

    #期待値を更新
    uv2 = np.repeat(0.0, hhpt)
    for j in range(get):
        index = index_history[j]
        uv2[index] += np.dot(theta_user2[index, ] * theta_v2[item_get[j], ], k_vec) * gamma_dt[index, j]


    ##トリガーのアイテム特徴行列をサンプリング
    #パラメータの格納用配列
    theta_dt = np.zeros((hhpt, k))
    uv_dt = np.zeros((hhpt, k))
    H21 = np.zeros((item, k))
    H22 = np.zeros((item, k))

    ##アイテムごとの事後分布のパラメータ
    for i in range(item):
        #補助変数を更新
        for j in range(get):
            index = item_list21[i][j]
            theta_target = theta_user2[index, ]
            uv_target = (theta_target * theta_v2[i, ]) * gamma_dt[index, j][:, np.newaxis]
            theta_dt[index, ] += uv_target / theta_v2[i, ] 
            uv_dt[index, ] += uv_target
        index = item_list22[i]
        Lambda = uv_dt[index, ] / mu[index, np.newaxis]

        #ガンマ分布の事後分布のパラメータ　
        lambda_y = Lambda * y_vec[index]
        H21[i, ] = np.sum(lambda_y, axis=0)
        H22[i, ] = np.sum(theta_dt[index, ], axis=0)
        theta_dt[index, ] = 0; uv_dt[index, ] = 0
    H21 = H21 + alpha; H22 = H22 + beta

    #パラメータをサンプリング
    theta_v2 = np.random.gamma(H21.reshape(-1), 1/H22.reshape(-1), item*k).reshape(item, k)

    #期待値を更新
    uv2 = np.repeat(0.0, hhpt)
    for j in range(get):
        index = index_history[j]
        uv2[index] += np.dot(theta_user2[index, ] * theta_v2[item_get[j], ], k_vec) * gamma_dt[index, j]


    ##トリガーの重みパラメータをサンプリング
    #補助変数を更新
    mu = uv1 + uv2
    par_dt = np.repeat(0.0, hhpt)
    weights_dt = np.repeat(0.0, hhpt)
    for j in range(get):
        index = index_history[j]
        theta_target = theta_user2[index, ]
        uv_target = np.dot(theta_target * theta_v2[item_get[j], ], k_vec) * gamma_dt[index, j]
        par_dt[index] += uv_target / gamma_vec[index]
        weights_dt[index] += uv_target
    Lambda = weights_dt / mu

    #ガンマ分布のパラメータ
    lambda_y = Lambda * y
    G1 = np.repeat(0.0, hh); G2 = np.repeat(0.0, hh)
    for i in range(hh):
        index = d_list[i]
        G1[i] = np.sum(lambda_y[index])
        G2[i] = np.sum(par_dt[index])
    G1 = G1 + alpha; G2 = G2 + beta

    #パラメータをサンプリング
    gamma = np.random.gamma(G1, 1/G2, hh)
    gamma_vec = gamma[d_id]

    #期待値を更新
    gamma_dt = np.zeros((hhpt, get))
    uv2 = np.repeat(0.0, hhpt)
    for j in range(get):
        index = index_history[j]
        gamma_dt[index, j] = gamma_vec[index] * z_get[j]
        uv2[index] += np.dot(theta_user2[index, ] * theta_v2[item_get[j], ], k_vec) * gamma_dt[index, j]
    mu = uv1 + uv2


    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        THETA_U1[:, :, mkeep] = theta_u1
        THETA_V1[:, :, mkeep] = theta_v1
        THETA_U2[:, :, mkeep] = theta_u2
        THETA_V2[:, :, mkeep] = theta_v2
        GAMMA[mkeep, ] = gamma

    if rp%disp==0:
        #対数尤度を更新
        mu = uv1 + uv2 
        LL = np.sum(scipy.stats.expon.logpdf(x=y, scale=mu))

        #サンプリング結果の表示
        print(rp)
        print(np.round(np.array([LL, LLst, LLbest]), 1))

0
[-1696541.4 -1836956.  -1460546.9]
10
[-1577842.7 -1836956.  -1460546.9]
20
[-1556162.1 -1836956.  -1460546.9]
30
[-1544699.5 -1836956.  -1460546.9]
40
[-1536898.3 -1836956.  -1460546.9]
50
[-1530281.3 -1836956.  -1460546.9]
60
[-1524311.6 -1836956.  -1460546.9]
70
[-1518572.4 -1836956.  -1460546.9]
80
[-1513276.3 -1836956.  -1460546.9]
90
[-1508957.3 -1836956.  -1460546.9]
100
[-1504566.9 -1836956.  -1460546.9]
110
[-1500813.2 -1836956.  -1460546.9]
120
[-1497512.5 -1836956.  -1460546.9]
130
[-1494395.1 -1836956.  -1460546.9]
140
[-1491222.  -1836956.  -1460546.9]
150
[-1488402.5 -1836956.  -1460546.9]
160
[-1485774.  -1836956.  -1460546.9]
170
[-1483680.8 -1836956.  -1460546.9]
180
[-1482143.8 -1836956.  -1460546.9]
190
[-1480819.7 -1836956.  -1460546.9]


KeyboardInterrupt: 